In [2]:
import numpy as np
import cv2
import playsound #1.2.2 
# import winsound

classes = []
f = open('/home/qwer/coco.names.txt', 'r')
classes = [line.strip() for line in f.readlines()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

yolo_model = cv2.dnn.readNet(
    '/home/qwer/yolov3.cfg', '/home/qwer/yolov3')  # 욜로 읽어오기
layer_names = yolo_model.getLayerNames()
output_layers = [layer_names[i[0] - 1]
    for i in yolo_model.getUnconnectedOutLayers()]


def process_video():  # 비디오에서 침입자 검출해 알리기
   video = cv2.VideoCapture('nvarguscamerasrc ! video/x-raw(memory:NVMM), width=416, height=416, format=(string)NV12, framerate=(fraction)30/1 ! nvvidconv ! video/x-raw, format=(string)BGRx ! videoconvert ! video/x-raw, format=(string)BGR ! appsink', cv2.CAP_GSTREAMER)
   while video.isOpened():
       success, img = video.read()
       if success:
           height, width, channels = img.shape
           blob = cv2.dnn.blobFromImage(
               img, 1.0/256, (416, 416), (0, 0, 0), swapRB=True, crop=False)

           yolo_model.setInput(blob)
           output3 = yolo_model.forward(output_layers)

           class_ids, confidences, boxes = [], [], []
           for output in output3:
               for vec85 in output:
                   scores = vec85[5:]
                   class_id = np.argmax(scores)
                   confidence = scores[class_id]
                   if confidence > 0.2:  # 신뢰도가 50% 이상인 경우만 취함
                       # [0,1] 표현을 영상 크기로 변환
                       centerx, centery = int(
                           vec85[0]*width), int(vec85[1]*height)
                       w, h = int(vec85[2]*width), int(vec85[3]*height)
                       x, y = int(centerx-w/2), int(centery-h/2)
                       boxes.append([x, y, w, h])
                       confidences.append(float(confidence))
                       class_ids.append(class_id)

           indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

           for i in range(len(boxes)):
               if i in indexes:
                   x, y, w, h = boxes[i]
                   text = str(classes[class_ids[i]])+'%.3f' % confidences[i]
                   cv2.rectangle(img, (x, y), (x+w, y+h),
                                 colors[class_ids[i]], 2)
                   cv2.putText(img, text, (x, y+30),
                               cv2.FONT_HERSHEY_PLAIN, 2, colors[class_ids[i]], 2)


           cv2.imshow('Object detection',img)

           if 0 in class_ids:
                print('보행자 인식')  
                #winsound.Beep(frequency=3000, duration=60) # 길게 하면 영상이 지연 댐 
                playsound.playsound('beep1.mp3')
                
                # cv2.imwrite("%s_%05d_%3d.jpg" % (filepath, i, 0), img) # 이미지 저장
                # i += 1 # 이미지 저장
                
           elif 1 in class_ids:
                print('차량 접근')
                # winsound.Beep(frequency=2000, duration=500)
                # playsound.playsound('car.mp3')

           elif 2 in class_ids:
                print('자전거 접근')
                #winsound.Beep(frequency=2000, duration=500)

           elif 3 in class_ids:
                print('오토바이 접근')
                #winsound.Beep(frequency=2000, duration=500)

           elif 4 in class_ids:  # 자전거
                print('전동킥보드 접근')
                #winsound.Beep(frequency=2000, duration=500)

           if 0 in class_ids and 1 in class_ids:
                print('차량 접근 알림')
                # winsound.Beep(frequency=5000, duration=20)
                playsound.playsound('beep2.mp3')
           if 0 in class_ids and 2 in class_ids:
                print('자전거 접근 알림')
                #winsound.Beep(frequency=5000, duration=700)

           if 0 in class_ids and 3 in class_ids:
                print('오토바이 접근 알림')
                #winsound.Beep(frequency=5000, duration=1500)

           if 0 in class_ids and 4 in class_ids:
                print('킥보드 접근 알림')
                #winsound.Beep(frequency=5000, duration=900)

            # else :
                # print('인식 대기중') # 작동 확인용
       key=cv2.waitKey(1) & 0xFF
       if key==27: break

   video.release()
   cv2.destroyAllWindows()

process_video()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 49)